# Ayse Dogan
## Creating splits for 2-weeks datasets
### 02/20/25 - Project3

In [ ]:
# Load the modules
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
import pickle
import numpy as np

In [ ]:
#predefined the file names for e4 and hex
#list(hex_data_dict[33]['day0_hex']['csv'].keys())+list(hex_data_dict[33]['day0_hex']['wav'].keys())
e4_file_names = ['TEMP.csv', 'tags.csv', 'HR.csv', 'ACC.csv', 'IBI.csv', 'EDA.csv', 'BVP.csv']
hex_file_names = ['RR_interval_quality.csv', 'NN_interval.csv', 'statistics.csv', 'sleep_position.csv', 'PTT.csv', 'step.csv', 'inspiration.csv', 'RR_interval.csv', 'expiration.csv', 
                  'device_position.csv', 'heart_rate.wav', 'heart_rate_quality.wav', 'systolic_pressure.wav', 'tidal_volume_adjusted.wav', 'minute_ventilation.wav', 'tidal_volume.wav', 
                  'respiration_abdominal.wav', 'acceleration_X.wav', 'acceleration_Y.wav', 'respiration_thoracic.wav', 'acceleration_Z.wav', 'ECG_I.wav', 'minute_ventilation_adjusted.wav', 
                  'ECG_III.wav', 'energy_mifflin_keytel.wav', 'systolic_pressure_adjusted.wav', 'breathing_rate_quality.wav', 'ECG_II.wav', 'cadence.wav', 'breathing_rate.wav', 'activity.wav']

In [ ]:
# Location of the Datasets
#parent_path = '/Users/adogan/Library/CloudStorage/Box-Box/[Box Health - Internal] Rad-Wear/[Box Health - Internal] WEAR data collection/WEAR Observation Peroid'
parent_path = '/work/hdd/bbnp/wear_raw/WEAR Observation Peroid'
participant_list = os.listdir(parent_path)
# Filter the list for entries that start with 'WEAR' and do not end with 'alias'
filtered_participant_list = [name for name in participant_list if name.startswith('WEAR') and not name.endswith('alias')]
print("The number of participants is: {}".format(len(filtered_participant_list)))
#filtered_participant_list

In [ ]:
order = 0
def get_participant_id(order):
    participant_id = int(filtered_participant_list[order][filtered_participant_list[order].index(' ')+2:])
    return(participant_id)

def generate_participant_id_list():
    participant_id_list = []
    for i in range(len(filtered_participant_list)):
        id = get_participant_id(i)
        participant_id_list.append(id)
    return(participant_id_list)
participant_id_list = generate_participant_id_list()

## Get the participant informations from the tracking data

In [ ]:
# Path to the Excel file
excel_file_path = '/Users/adogan/Library/CloudStorage/Box-Box/[Box Health - Internal] Rad-Wear/[Box Health - Internal] WEAR data collection/WEAR participant tracking.xlsx'
excel_file_path = '/work/hdd/bbnp/wear_raw/WEAR participant tracking.xlsx'
all_sheets = pd.read_excel(excel_file_path, sheet_name=None)

# The dictionary now contains sheet names as keys and their DataFrames as values
all_sheets_keys = all_sheets.keys()  # Display all sheet names
participants_sheets_keys = [name for name in all_sheets_keys if name.startswith('P') and not name.startswith('Participants') ]
#participants_sheets_keys

In [ ]:
# Create a new dictionary with only the filtered keys
participants_sheets_dict = {
    name: all_sheets[name]  # Keep the original key-value pair
    for name in participants_sheets_keys  # Use your filtered list of keys
}

# Rename keys by extracting integers after 'P'
participants_sheets_dict = {
    int(name[1:]): participants_sheets_dict[name]  # Extract the integer after 'P' and set it as the key
    for name in participants_sheets_keys  # Use your filtered sheet names list
}

# Display the new dictionary keys
print("The number of participants in the tracking list: {}".format(len(set(list(participants_sheets_dict.keys())))))
print(participants_sheets_dict.keys())

#unwanted_numbers = {2, 223, 249, 252, 253, 264, 268, 290} #why we subtracted them I have no idea #2/20/25
wanted_numbers = set(list(participants_sheets_dict.keys())) & set(participant_id_list)
# Filter the participant list to exclude matching numbers
filtered_participant_list = [
    name for name in filtered_participant_list
    if name.split(' ')[-1][1:].isdigit() and int(name.split(' ')[-1][1:]) in wanted_numbers
]

In [ ]:
#get the paths for the patients
def get_the_paths_to_patient_folders(order, parent_path = parent_path, filtered_participant_list =filtered_participant_list):
    participant_folder = parent_path+'/'+filtered_participant_list[order]
    folder_list_of_patient = os.listdir(participant_folder)
    folder_list_of_patient = [folder for folder in folder_list_of_patient if folder != '.DS_Store']
    return(participant_folder)

# get_the_paths_to_patient_folders(order)

In [ ]:
#read the participants data in the recorded excels
def get_device_day_details(order,participants_sheets_dict=participants_sheets_dict):
    # Access the DataFrame for the specific participant
    #print('participant_id:', get_participant_id(order))
    participant_patients_dayly_tags = participants_sheets_dict[get_participant_id(order)]
    
    # Set the 3rd row (index 2) as the header
    participant_patients_dayly_tags.columns = participant_patients_dayly_tags.iloc[3,:].tolist()  # Set the 3rd row as column headers
    participant_patients_dayly_tags = participant_patients_dayly_tags[4:]  # Remove the first 3 rows to clean the data
    
    # Reset the index for clean DataFrame
    participant_patients_dayly_tags.reset_index(drop=True, inplace=True)
    
    return participant_patients_dayly_tags
participant_patients_dayly_tags = get_device_day_details(order)
#participant_patients_dayly_tags

In [ ]:
# name each day to create a map
def generate_dataset_dict(order):
    participant_patients_dayly_tags = get_device_day_details(order)
    # Initialize an empty dictionary
    data_dict = {}
    
    # Iterate through the rows of the DataFrame
    for index, row in participant_patients_dayly_tags.iterrows():
        # Format the keys
        e4_key = f"day{index:1}_e4"  # Key for E4 (e.g., day00_e4)
        hex_key = f"day{index:1}_hex"  # Key for Hexoskin (e.g., day00_hex)
        
        # Assign values from the DataFrame to the dictionary
        data_dict[e4_key] = row[participant_patients_dayly_tags.columns[1]]  # Value from the E4 column
        data_dict[hex_key] = row[participant_patients_dayly_tags.columns[2]]  # Value from the Hexoskin column
    
    # Print the resulting dictionary
    return(data_dict)

participant_folder_list = generate_dataset_dict(order)
#participant_folder_list

In [ ]:
#Read each participant base ond [key][day] ==> [order][file_code]

datasets = ['ECG_I.wav','breathing_rate.wav','acceleration_X.wav','acceleration_Y.wav','acceleration_Z.wav', ]
def generate_selected_dataset(hex_data_dict,datasets = datasets):
    e4_BVP_dict = {}
    for key, directory in hex_data_dict.items():
        e4_BVP_dict[key] = {}
        for day, read_files in directory.items():
            e4_BVP_dict[key][day] = {}
            read_files_list = list(read_files)
            #print("read_files_list:", read_files_list)
            for dataset in datasets:
                if dataset in read_files_list:
                    # print('key:', key, 'day:', day, 'is tagged.')
                    e4_BVP_dict[key][day][dataset] = hex_data_dict[key][day][dataset]
                else:
                    print('key:', key, 'day:', day, 'is empty.')
                    e4_BVP_dict[key][day][dataset] = {}
    return(e4_BVP_dict)

from scipy.io import wavfile
def generate_data_dict_for_wav_files(order, file_code = 'day0_hex', csv_files =datasets):
    participant_folder = get_the_paths_to_patient_folders(order)
    participant_folder_list = generate_dataset_dict(order)
    target = participant_folder+'/'+participant_folder_list[file_code]
    # Get the list of files in the directory
    #file_list_of_patient_day_device = os.listdir(target)
    
    # Filter to include only CSV files
    #csv_files = [file for file in file_list_of_patient_day_device if file.endswith('.wav')]
    #print('csv_files:', csv_files)
    
    # Create a dictionary to store the data from CSV files
    csv_data_dict = {}
    
    # Loop through the CSV files and read them into the dictionary
    for csv_file in csv_files:
        csv_path = os.path.join(target, csv_file)  # Construct full path to the CSV file
        print('csv_path: ', csv_path)
        try:
            # Check if the file is empty
            if os.stat(csv_path).st_size > 0:  # File is non-empty
                sample_rate, data = wavfile.read(csv_path)
                #print('sample_rate:',sample_rate, 'for file: ', csv_file)
                csv_data_dict[csv_file] = data
            else:
                print(f"Skipping empty file: {csv_file}")
        except Exception as e:
            print(f"Error reading {csv_file}: {e}")
    
    # Display the keys of the dictionary (file names) and optionally the first few rows of one DataFrame
    #print(csv_data_dict.keys())  # Keys will be the file names (e.g., 'TEMP.csv', 'EDA.csv')
    return(csv_data_dict)
from pathlib import Path
from scipy.io import wavfile

def generate_data_dict_for_wav_files(order,
                                     file_code='day0_hex',
                                     wav_names=datasets):   # list of expected files
    """
    Return {wav_name: np.ndarray} for every existing, non‑empty WAV in wav_names.
    Missing or empty files are skipped with a message.
    """
    # ------- build the day directory path safely -----------------------------
    base_dir   = Path(get_the_paths_to_patient_folders(order))
    day_folder = base_dir / generate_dataset_dict(order)[file_code]

    if not day_folder.is_dir():
        #print(f"[missing folder] {order}")
        return {}

    wav_dict = {}

    for name in wav_names:
        path = day_folder / name                # Path object ⇒ OS‑safe join

        # ---------- new existence check --------------------------------------
        if not path.is_file():                  # False for missing or a directory
           # print(f"[missing file] {name}")
            continue

        # ---------- size check ------------------------------------------------
        if path.stat().st_size == 0:
            #print(f"[empty file]   {name}")
            continue

        # ---------- safe read -------------------------------------------------
        try:
            sr, data = wavfile.read(path)
            wav_dict[name] = data
        except Exception as e:
            print(f"[error reading] {name}: {e}")

    return wav_dict

# record all the e4 readings in e4_data_dict: e4_data_dict[id][dayX_e4][<e4 file name>]
def hex_data_reading():
    hex_data_dict = {}
    for order in range(len(filtered_participant_list)):
    #for order in range(3):
        # Process each 'e4' key
        participant_folder_list = generate_dataset_dict(order)
        # Get keys ending with 'e4' and 'hex'
        #e4_keys = [key for key in participant_folder_list.keys() if key.endswith('e4')]
        hex_keys = [key for key in participant_folder_list.keys() if key.endswith('hex')]
        key = get_participant_id(order)
        #print('key: ', key)
        hex_data_dict[key]= {}
        for dayX_e4 in hex_keys:
            try:
                # Generate the data for the current 'e4' key
                #hex_data_dict[key][dayX_e4] = generate_data_dict(order, file_code=dayX_e4)
                #print('Participant',key, dayX_e4)
                hex_data_dict[key][dayX_e4] = generate_data_dict_for_wav_files(order, file_code=dayX_e4)
            except Exception as e:
                print(f"Error processing {dayX_e4} for participant {key}")
                #hex_data_dict[key][dayX_e4] = {}

    # # Display the keys and optionally inspect the data
    # print(e4_data_dict.keys())
    # # Example: print data for a specific key
    # if 'day0_e4' in e4_data_dict:
    #     print(e4_data_dict['day0_e4'])
    return hex_data_dict


#hex_selected_dict = hex_data_reading()#hex_data_dict, datasets)


In [ ]:
def visualize_daily_data_of_participant(hex_selected_dict, id = 92, day='day0_hex'):
    # Assuming e4_BVP_dict[33] is a dictionary where keys are days and values are DataFrames or Series
    participant_data = hex_selected_dict[id][day]['ECG_I.wav']  # Replace 33 with the key of the participant you want to analyze

    # Loop through each day in the participant's data
    plt.figure(figsize=(12, 6))
    # for day, data in participant_data.items():
    #     print('day', day)
    #     print('data:', data)
    #     if len(data) == 0:
    #         print("{day} is empty!")
    #     else:
    #         #plt.figure(figsize=(12, 6))
            
    #         # Plot the data for the day
    plt.plot(participant_data, color='blue', linewidth=0.5)
    
    # Customizing the plot
    plt.title(f'Data for participant {id} {day}', fontsize=16)
    plt.xlabel('Time (samples)', fontsize=14)
    plt.ylabel('BVP Value', fontsize=14)
    plt.grid(alpha=0.5)
    
    # Show the plot
    plt.tight_layout()
    #plt.show()


# E4 Reading

In [ ]:
# record all the e4 readings in e4_data_dict: e4_data_dict[id][dayX_e4][<e4 file name>]
#Read each participant base ond [key][day] ==> [order][file_code]
def generate_data_dict(order, file_code = 'day0_e4', file_type = 'TEMP'):
    participant_folder = get_the_paths_to_patient_folders(order)
    participant_folder_list = generate_dataset_dict(order)
    target = participant_folder+'/'+participant_folder_list[file_code]
    # Get the list of files in the directory
    file_list_of_patient_day_device = os.listdir(target)
    
    # Filter to include only CSV files
    
    if file_type == 'TEMP':
        csv_files = [file for file in file_list_of_patient_day_device if file.startswith('TEMP.csv') or file.startswith('tags.csv')]
    else:
        csv_files = [file for file in file_list_of_patient_day_device if file.endswith('.csv')]
    # Create a dictionary to store the data from CSV files
    csv_data_dict = {}
    
    # Loop through the CSV files and read them into the dictionary
    for csv_file in csv_files:
        csv_path = os.path.join(target, csv_file)  # Construct full path to the CSV file
        try:
            # Check if the file is empty
            if os.stat(csv_path).st_size > 0:  # File is non-empty
                csv_data_dict[csv_file] = pd.read_csv(csv_path)
            else:
                print(f"Skipping empty file: {csv_file}")
        except Exception as e:
            print(f"Error reading {csv_file}: {e}")
    
    # Display the keys of the dictionary (file names) and optionally the first few rows of one DataFrame
    #print(csv_data_dict.keys())  # Keys will be the file names (e.g., 'TEMP.csv', 'EDA.csv')
    return(csv_data_dict)


def e4_data_reading(file_type = 'TEMP'):
    e4_data_dict = {}
    for order in range(len(filtered_participant_list)):
    #for order in range(3):
        # Process each 'e4' key
        participant_folder_list = generate_dataset_dict(order)
        # Get keys ending with 'e4' and 'hex'
        e4_keys = [key for key in participant_folder_list.keys() if key.endswith('e4')]
        # hex_keys = [key for key in participant_folder_list.keys() if key.endswith('hex')]
        key = get_participant_id(order)
        #print('key: ', key)
        e4_data_dict[key]= {}
        for dayX_e4 in e4_keys:
            try:
                # Generate the data for the current 'e4' key
                e4_data_dict[key][dayX_e4] = generate_data_dict(order, file_code=dayX_e4,file_type = file_type)
                
            except Exception as e:
                print(f"Error processing {dayX_e4} for participant {key}")
                e4_data_dict[key][dayX_e4] = {}

    # # Display the keys and optionally inspect the data
    # print(e4_data_dict.keys())
    # # Example: print data for a specific key
    # if 'day0_e4' in e4_data_dict:
    #     print(e4_data_dict['day0_e4'])
    return e4_data_dict


In [ ]:
def add_time_column(beginning_time,selected_dict_to_add_column , fs_ecg =256,fs_br = 1,fs_acc = 64):
    #participant_dict_HR ={}
    #day_num = day[3]

    '''ECG processing'''
    #raw_ECG = hex_selected_dict[participant][day]['ECG_I.wav']
    raw_ECG = selected_dict_to_add_column['ECG_I.wav']
    raw_ECG = pd.DataFrame(data = raw_ECG)
    ecg = 0.0064 * raw_ECG #get correct magnitude of ECG
    ecg.rename(columns = {0: 'ECG'}, inplace = True)
    #date_info = e4_data_dict[participant]['day'+day_num+'_e4']['TEMP.csv'].columns[0]
    #ecg
    t0_ecg = beginning_time
    #ecg = participant_info_hex['raw_ECG']

    ecg['Timestamp'] = list(range(0, ecg.shape[0],1))
    ecg['Timestamp'] = ecg['Timestamp'].apply(lambda x: x/fs_ecg+t0_ecg)
    #ecg['Timestamp'] = ecg['Timestamp'].str.get(0)
    # ecg['Second'] = ecg['Timestamp']
    ecg = ecg.set_index('Timestamp')
    # ecg['Second'] = ecg['Second'].apply(lambda x: x-ecg.index[0])
    ecg = ecg.reset_index()
    ecg.rename(columns={0: 'ecg'}, inplace=True)


    '''BR processing'''
    #raw_br = hex_selected_dict[participant][day]['breathing_rate.wav'] # wavfile.read(os.path.join(ecg_folder, subfolder, "breathing_rate.wav"))
    raw_br = selected_dict_to_add_column['breathing_rate.wav']
    raw_br = pd.DataFrame(data = raw_br)
    br = 1.0000 * raw_br
    br.rename(columns = {0: 'breathing_rate'}, inplace = True)
    
    #breath rate
    t0_br = beginning_time
    #br = participant_info_hex['raw_br']
    br['Timestamp'] = list(range(0, br.shape[0],1))
    br['Timestamp'] = br['Timestamp'].apply(lambda x: x/fs_br+t0_br)
    # br['Second'] = br['Timestamp']
    br = br.set_index('Timestamp')
    # br['Second'] = br['Second'].apply(lambda x: x-br.index[0])
    br = br.reset_index()
    br.rename(columns={0: 'br'}, inplace=True)


    '''ACC processing'''
    #raw_accX = wavfile.read(r'/home/maxinehe/Downloads/'+fhex+'/acceleration_X.wav')
    #raw_accX = hex_selected_dict[participant][day]['acceleration_X.wav'] #wavfile.read(os.path.join(ecg_folder, subfolder, "acceleration_X.wav"))
    raw_accX = selected_dict_to_add_column['acceleration_X.wav']
    raw_accX = pd.DataFrame(data = raw_accX)
    accx = 1.0000 * raw_accX 

    #raw_accY = wavfile.read(r'/home/maxinehe/Downloads/'+fhex+'/acceleration_Y.wav')
    #raw_accY = hex_selected_dict[participant][day]['acceleration_Y.wav'] #wavfile.read(os.path.join(ecg_folder, subfolder, "acceleration_Y.wav"))
    raw_accY = selected_dict_to_add_column['acceleration_Y.wav']
    raw_accY = pd.DataFrame(data = raw_accY)
    accy = 1.0000 * raw_accY 

    #raw_accZ = wavfile.read(r'/home/maxinehe/Downloads/'+fhex+'/acceleration_Z.wav')
    #raw_accZ = hex_selected_dict[participant][day]['acceleration_Z.wav'] #wavfile.read(os.path.join(ecg_folder, subfolder, "acceleration_Z.wav"))
    raw_accZ = selected_dict_to_add_column['acceleration_Z.wav']
    raw_accZ = pd.DataFrame(data = raw_accZ)
    accz = 1.0000 * raw_accZ 

    raw_acc = pd.concat([accx, accy, accz], axis=1, ignore_index=True)
    raw_acc.rename(columns = {0: 'ACC_X', 1: 'ACC_Y', 2: 'ACC_Z'}, inplace = True)

    

    #acceleration
    t0_acc = beginning_time
    #raw_acc = participant_info_hex['raw_acc']
    raw_acc['Timestamp'] = list(range(0, raw_acc.shape[0],1))
    raw_acc['Timestamp'] = raw_acc['Timestamp'].apply(lambda x: x/fs_acc+t0_acc)
    # raw_acc['Second'] = raw_acc['Timestamp']
    raw_acc = raw_acc.set_index('Timestamp')
    # raw_acc['Second'] = raw_acc['Second'].apply(lambda x: x-raw_acc.index[0])
    raw_acc = raw_acc.reset_index()
    data_dict = {'processed_ECG':ecg, 'processed_br':br, 'processed_acc':raw_acc}
    return(data_dict)


## Generate Hex data with time and read e4 data

In [ ]:
# hex_selected_dict = hex_data_reading()
# e4_data_dict = e4_data_reading(file_type = 'TEMP')
# participants = list(hex_selected_dict.keys())
# fs_ecg = 256
# hex_processed_dict ={}
# for participant in participants:
#     if len([k for k, v in hex_selected_dict[participant].items() if v])>0: #to get rid of 32
#         days = hex_selected_dict[participant].keys()
#         hex_processed_dict[participant]={}
#         for day in days:
#             day_num = day[3]
#             if len(e4_data_dict[participant]['day'+day_num+'_e4'].keys())>0:
#                 date_info = e4_data_dict[participant]['day'+day_num+'_e4']['TEMP.csv'].columns[0]
#                 beginning_time = float(date_info)#/fs_ecg
#                 selected_dict_to_add_column = hex_selected_dict[participant][day]
#                 if len(selected_dict_to_add_column.keys())== 5:
#                     hex_processed_dict[participant][day] = {}
#                     #print('participant:',participant, day)
#                     hex_processed_dict[participant][day] = add_time_column(beginning_time, selected_dict_to_add_column)
            #     else: 
            #         #print('Sensor signals are not complete:', list(selected_dict_to_add_column.keys()), 'participant: ',participant, 'day'+day_num+'_hex')
            #         #print('participant: ',participant, 'day'+day_num+'_hex is empty in hex_selected_dict!')
            # else:
            #     print('participant: ',participant, 'day'+day_num+'_e4 is empty in e4_data_dict!')

## Syncranization for E4

In [ ]:
def find_closest_index(timestamps, target):

    idx = (np.abs(timestamps - target)).argmin()
    return idx

def internalsync_offset(eda, temp, bvp, acc):
    # Extract timestamps to numpy arrays for efficient operations
    ts_eda = eda['Timestamp'].values
    ts_temp = temp['Timestamp'].values
    ts_bvp = bvp['Timestamp'].values
    ts_acc = acc['Timestamp'].values
    
    # Determine the common time frame across all signals
    start_time = max(ts_eda[0], ts_temp[0], ts_bvp[0], ts_acc[0])
    end_time = min(ts_eda[-1], ts_temp[-1], ts_bvp[-1], ts_acc[-1])
    
    # Synchronize each signal to the common time frame
    offsets = {}
    for name, ts in [('eda', ts_eda), ('temp', ts_temp), ('bvp', ts_bvp), ('acc', ts_acc)]:
        start_idx = find_closest_index(ts, start_time)
        end_idx = find_closest_index(ts, end_time) + 1  # Include the end index in the slice
        
        if name == 'eda':
            offsets['eda'] = eda.iloc[start_idx:end_idx]
        elif name == 'temp':
            offsets['temp'] = temp.iloc[start_idx:end_idx]
        elif name == 'bvp':
            offsets['bvp'] = bvp.iloc[start_idx:end_idx]
        elif name == 'acc':
            offsets['acc'] = acc.iloc[start_idx:end_idx]
    
    return offsets['eda'], offsets['temp'], offsets['bvp'], offsets['acc']

def generate_offset_e4(participant_dict):
#     participant_dict = import_e4_csv_files(root_dir =root_dir)
    participant_dict_e4_offset = {}
#    print('participant_dict: ',participant_dict.keys())
    print("list(participant_dict.keys(): ",list(participant_dict.keys()))
    for j in range(len(list(participant_dict.keys()))):
        print('j:', j)
        #print('list(participant_dict.keys())[j]: ', list(participant_dict.keys())[j])
        selected_key = list(participant_dict.keys())[j]
        print("selected_key:", selected_key)
        a = participant_dict[selected_key]
        print('a: ', a)
        print("a.keys:", a.keys())
        # offset_eda, offset_temp, offset_bvp, offset_acc_e4 = internalsync_offset(a['EDA'],a['TEMP'],a['BVP'],a['ACC'])
        offset_eda, offset_temp, offset_bvp, offset_acc_e4 = internalsync_offset(a['EDA.csv'],a['TEMP.csv'],a['BVP.csv'],a['ACC.csv'])
        data_dict = {'offset_eda':offset_eda, 'offset_temp':offset_temp, 'offset_bvp':offset_bvp,"offset_acc_e4":offset_acc_e4 }
        participant_dict_e4_offset[list(participant_dict.keys())[j]] = data_dict
    return(participant_dict_e4_offset)

#participant_dict_e4_offset = generate_offset_e4(e4_data_dict[33])

## Syncranization for Hex

In [ ]:
def generate_offset_hex(participant, hex_processed_dict ):
#     participant_dict_HR = import_hex_signals(root_dir =root_dir,fhex = fhex)
    participant_dict_hex_offset = {}
    for j in range(len(list(hex_processed_dict[participant].keys()))):
        a = hex_processed_dict[participant][list(hex_processed_dict[participant].keys())[j]]
        
        if a['processed_ECG'].shape[0] != 0:
            offset_ecg, offset_br, offset_acc_hex, offset_acc_hex = internalsync_offset(a['processed_ECG'],a['processed_br'],a['processed_acc'], a['processed_acc'])
            data_dict = {'offset_ecg':offset_ecg, 'offset_br':offset_br, 'offset_acc_hex':offset_acc_hex}
            participant_dict_hex_offset[list(hex_processed_dict[participant].keys())[j]] = data_dict
        else: 
            print(participant, list(hex_processed_dict[participant].keys())[j], 'ECG data is empty!')
    return(participant_dict_hex_offset)


# # Constants
# sampling_rate = 256  # Hz
# duration_per_split = 30 #min
# participant_dict_hex_offset ={}
# for participant in hex_processed_dict.keys():
#     #print('participant: ', participant)
#     participant_dict_hex_offset[participant] = generate_offset_hex(participant, hex_processed_dict=hex_processed_dict)

In [ ]:
sampling_rate = 256  # Hz
duration_per_split = 5 #min
participant_dict_hex_offset ={}

hex_selected_dict = hex_data_reading()
e4_data_dict = e4_data_reading(file_type = 'TEMP')
participants = list(hex_selected_dict.keys())
fs_ecg = 256
hex_processed_dict ={}
for participant in participants:
    if len([k for k, v in hex_selected_dict[participant].items() if v])>0: #to get rid of 32
        days = hex_selected_dict[participant].keys()
        hex_processed_dict[participant]={}
        for day in days:
            day_num = day[3]
            if len(e4_data_dict[participant]['day'+day_num+'_e4'].keys())>0:
                date_info = e4_data_dict[participant]['day'+day_num+'_e4']['TEMP.csv'].columns[0]
                beginning_time = float(date_info)#/fs_ecg
                selected_dict_to_add_column = hex_selected_dict[participant][day]
                if len(selected_dict_to_add_column.keys())== 5:
                    hex_processed_dict[participant][day] = {}
                    #print('participant:',participant, day)
                    hex_processed_dict[participant][day] = add_time_column(beginning_time, selected_dict_to_add_column)
        print(participant)
        participant_dict_hex_offset[participant] = generate_offset_hex(participant, hex_processed_dict)

# for participant in hex_processed_dict.keys():
#     #print('participant: ', participant)
#     participant_dict_hex_offset[participant] = generate_offset_hex(participant, hex_processed_dict=hex_processed_dict)

In [ ]:
def save_pickles(participant_dict_hex_offset):


    # Save all_split_dict
    with open('participant_dict_hex_offset.pkl', 'wb') as f:
        pickle.dump(participant_dict_hex_offset, f)

    
    hex_ECG_dict_path = os.path.abspath('participant_dict_hex_offset.pkl')
    #label_dict_path = os.path.abspath('all_label_dict_'+name+'_min.pkl)
    print("The pickle files are saved in:\n ", hex_ECG_dict_path)

#save_pickles(participant_dict_hex_offset)

In [ ]:
def build_hex_ecg_dict(offset_dict, signal_key='offset_ecg'):
    """
    Extract the desired signal (default = 'offset_ecg') from a
    participant_dict_hex_offset–style structure.

    Parameters
    ----------
    offset_dict : dict
        Mapping      participant_id → day_key → {signal_key, …}
    signal_key   : str, optional
        The key whose value you want to keep (default 'offset_ecg').

    Returns
    -------
    hex_ecg_dict : dict
        Same two‑level layout (participant → day), but each day maps
        directly to the signal object (e.g. a NumPy array or DataFrame)
        instead of the whole modality bundle.
    """
    return {
        pid: {
            day: day_bundle[signal_key]
            for day, day_bundle in day_map.items()
            if signal_key in day_bundle          # skip if missing
        }
        for pid, day_map in offset_dict.items()
    }

# ---- usage ----------------------------------------------------------
hex_ECG_dict = build_hex_ecg_dict(participant_dict_hex_offset)

# # quick sanity‑check
# print(hex_ECG_dict.keys())                     # participants
# print(hex_ECG_dict[33].keys())                 # day0_hex … day9_hex
#      # whatever 'offset_ecg' holds


In [ ]:
# def save_pickles(hex_ECG_dict):


#     # Save all_split_dict
#     with open('hex_ECG_dict.pkl', 'wb') as f:
#         pickle.dump(hex_ECG_dict, f)

    
#     hex_ECG_dict_path = os.path.abspath('hex_ECG_dict.pkl')
#     #label_dict_path = os.path.abspath('all_label_dict_'+name+'_min.pkl)
#     print("The pickle files are saved in:\n ", hex_ECG_dict_path)

# save_pickles(hex_ECG_dict)

In [ ]:

def generate_split_dict(patients_dayly_tags, sampling_rate = sampling_rate, duration_per_split = duration_per_split):
    duration_per_split = duration_per_split * 60  # 30 minutes in seconds
    rows_per_split = int(duration_per_split * sampling_rate)  # Total rows for 30 minutes

    # Initialize the dictionary to store splits
    split_dict = {}

    # Split the dataframe into chunks of 30 minutes
    for i in range(math.ceil(len(patients_dayly_tags) / rows_per_split)):
        start_index = i * rows_per_split
        end_index = start_index + rows_per_split
        split_patients_dayly_tags = patients_dayly_tags.iloc[start_index:end_index]
        
        # Use the starting time of the split as the dictionary key
        split_start_time = split_patients_dayly_tags['Timestamp'].iloc[0]
        split_dict[split_start_time] = split_patients_dayly_tags.drop(columns=['Timestamp']).reset_index(drop=True)

    # Example: Access one of the dictionaries
    # print("Number of splits:", len(split_dict))
    # print("Sample split key:", list(split_dict.keys())[0])
    # print("Sample split dataframe:")
    # print(split_dict[list(split_dict.keys())[0]])
    return split_dict
# split_dict = generate_split_dict(patients_dayly_tags)

def generate_label_dict(tags,split_dict,duration_per_split = duration_per_split ):

    # Initialize label dictionary
    label_dict = {}

        # If tags length is 0, set all labels to 0
    if len(tags) == 0:
        for split_start_time in split_dict.keys():
            label_dict[split_start_time] = 0
        return label_dict

    # Create labels for each split in the split dictionary
    for split_start_time in split_dict.keys():
        # Calculate the 30-minute threshold (in seconds)
        split_end_time = split_start_time + duration_per_split * 60
        
        # Check if any tag falls within the 30-minute window of the current split
        label = int(any(tag >= split_start_time and tag < split_end_time for tag in tags.values.reshape(-1)))

        
        # Assign the label to the split
        label_dict[split_start_time] = label
    return label_dict



In [ ]:
# Assuming e4_data_dict is defined
def count_the_tags(e4_data_dict):
    e4_tags_dict = {}
    for key, directory in e4_data_dict.items():
        e4_tags_dict[key] = {}
        for day, read_files in directory.items():
            e4_tags_dict[key][day] = {}
            read_files_list = list(read_files)
            if 'tags.csv' in read_files_list:
                #print('key:', key, 'day:', day, 'is tagged.')
                e4_tags_dict[key][day] = e4_data_dict[key][day]['tags.csv']
            else:
                #print('key:', key, 'day:', day, 'is empty.')
                e4_tags_dict[key][day] = {}
    return(e4_tags_dict)

def generate_tag_counts_dict(e4_tags_dict):
    tag_counts = {}

    for key, days in e4_tags_dict.items():
        tag_counts[key] = {}
        for day, tags in days.items():
            if isinstance(tags, pd.DataFrame):  # Check if tags is a DataFrame
                tag_counts[key][day] = len(tags) if not tags.empty else 0
            else:
                tag_counts[key][day] = 0  # No tags or not a DataFrame
    # Finding keys with no tagging (all tag counts are 0)
    no_tagging_keys = [key for key, days in tag_counts.items() if all(count == 0 for count in days.values())]

    print("no_tagging_keys: {}".format(no_tagging_keys))
    return(tag_counts)


def generate_patients_dayly_tags_from_tag_counts(tag_counts):
    data = []
    for key, days in tag_counts.items():
        for day, count in days.items():
            data.append({'Key': key, 'Day': day, 'Tags': count})

    patients_dayly_tags = pd.DataFrame(data)
    return(patients_dayly_tags)


In [ ]:
def generate_pickle_for_data_dict(hex_ECG_dict,e4_tags_dict, sampling_rate = sampling_rate,duration_per_split =duration_per_split):
    all_split_dict = {}
    all_label_dict = {}
    for key in list(hex_ECG_dict.keys()):
        print('key: ', key)
        all_split_dict[key] = {}
        all_label_dict[key] = {}
        for day in hex_ECG_dict[key]:
            #print(day)
            if len(hex_ECG_dict[key][day])>0:
                #print('satisfied')
                #patients_dayly_tags[patients_dayly_tags['Key']==key]
                todays_tags = hex_ECG_dict[key][day]
                split_dict = generate_split_dict(todays_tags,sampling_rate = sampling_rate, duration_per_split = duration_per_split)
                #split_dict = generate_split_dict(patients_dayly_tags, sampling_rate = sampling_rate, duration_per_split = duration_per_split)
                day_num = day[3]
                tags = e4_tags_dict[key]['day'+day_num+'_e4']  
                label_dict = generate_label_dict(tags,split_dict,duration_per_split = duration_per_split )
                all_split_dict[key][day]=split_dict
                all_label_dict[key][day]=label_dict
            else:
                print(f"For participant {key} day {day} is empty!")
    return(all_split_dict,all_label_dict )


#e4_data_dict = e4_data_reading(file_type = 'TEMP')
e4_tags_dict = count_the_tags(e4_data_dict)
# tag_counts =generate_tag_counts_dict(e4_tags_dict)
# patients_dayly_tags = generate_patients_dayly_tags_from_tag_counts(tag_counts)

your_keys = [33, 8]
border = 5
#name_border='first_'+str(border)
your_keys = list(hex_ECG_dict.keys()) #[:border]
dict_you_want = {key: hex_ECG_dict[key] for key in your_keys}
all_split_dict,all_label_dict = generate_pickle_for_data_dict(dict_you_want,e4_tags_dict)   

In [ ]:
# def save_pickles(all_split_dict, all_label_dict, duration_per_split):
#     name = str(duration_per_split)

#     # Save all_split_dict
#     with open('all_split_ECG_dict_'+name+'_min.pkl', 'wb') as f:
#         pickle.dump(all_split_dict, f)

#     # Save all_label_dict
#     with open('all_label_ECG_dict_'+name+'_min.pkl', 'wb') as f:
#         pickle.dump(all_label_dict, f)
    
#     split_dict_path = os.path.abspath('all_split_ECG_dict_'+name+'_min.pkl')
#     label_dict_path = os.path.abspath('all_label_dict_'+name+'_min.pkl')
#     print("The pickle files are saved in:\n ", split_dict_path)

# save_pickles(all_split_dict, all_label_dict, duration_per_split)
import os
import pickle

def save_pickles(all_split_dict, all_label_dict, duration_per_split,
                 dest_dir="/work/hdd/bbnp/wear_raw"):
    name = str(duration_per_split)

    # Make sure the directory exists
    os.makedirs(dest_dir, exist_ok=True)

    # Build file paths
    split_filename = f"all_split_ECG_dict_{name}_min.pkl"
    label_filename = f"all_label_ECG_dict_{name}_min.pkl"
    split_path = os.path.join(dest_dir, split_filename)
    label_path = os.path.join(dest_dir, label_filename)

    # Save
    with open(split_path, 'wb') as f:
        pickle.dump(all_split_dict, f)
    with open(label_path, 'wb') as f:
        pickle.dump(all_label_dict, f)

    print("Pickle files saved to:")
    print("  ", split_path)
    print("  ", label_path)

# Usage:
save_pickles(all_split_dict, all_label_dict, duration_per_split)
